In [1]:
import torch
import torch.nn as nn
import random
import gym
import numpy as np
import cv2
from torch.utils.tensorboard import SummaryWriter
import statistics

# Preprocess each frame from 210x160 to 84x84
class Preprocess(gym.ObservationWrapper):
    
    def __init__(self, env=None):
        super(Preprocess, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return Preprocess.process(obs)

    def process(frame):
        frame = np.reshape(frame, [210, 160, 1]).astype(np.uint8)
        resized_frame = cv2.resize(frame, (84, 110), interpolation=cv2.INTER_AREA)
        result = resized_frame[18:102, :]
        result = np.reshape(result, [84, 84, 1])
        return result.astype(np.uint8)

# Change axis accordingly to neural network input
class ChangeAxis(gym.ObservationWrapper):
    
    def __init__(self, env):
        super(ChangeAxis, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)

C:\Users\User\anaconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\User\anaconda3\envs\torch_env\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [2]:
#Convolutional neural network
class DQN_Network(nn.Module):
    
    def __init__(self, input_shape, n_actions):
        super(DQN_Network, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        output_size = self.get_output_size(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(output_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    
    def get_output_size(self, input_shape):
        ot = self.conv(torch.zeros(1, *input_shape))
        return int(np.prod(ot.size()))
    
    def forward(self, x):
        x = x.float() / 255.0
        mid_output = self.conv(x).view(x.size()[0], -1)
        return self.fc(mid_output)

#DQN agent 
class DQN_Agent:

    def __init__(self, state_space, action_space, epsilon_max, epsilon_min, rb_size, batch_size, gamma, lr):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.state_space = state_space
        self.action_space = action_space        
       
        self.dqn = DQN_Network(state_space, action_space).to(self.device)
        self.dqn_target = DQN_Network(state_space, action_space).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.dqn.parameters(), lr=lr)
        self.l1 = nn.SmoothL1Loss().to(self.device)
        self.batch_size = batch_size
        self.gamma = gamma
        
        self.epsilon = epsilon_max
        self.epsilon_max = epsilon_max
        self.epsilon_min = epsilon_min
        self.epsilon_decay = (epsilon_max - epsilon_min)/20000

        self.mem_size = rb_size
        self.state_buffer = torch.zeros(self.mem_size, *self.state_space, dtype=torch.uint8)
        self.action_buffer = torch.zeros(self.mem_size, 1)
        self.reward_buffer = torch.zeros(self.mem_size, 1)
        self.next_state_buffer = torch.zeros(self.mem_size, *self.state_space, dtype=torch.uint8)
        self.done_buffer = torch.zeros(self.mem_size, 1)
        self.last_pos = 0
        self.current = 0
    
    def select_action(self, state):
        if random.random() < self.epsilon:  
            return torch.tensor([[random.randrange(self.action_space)]])
        else:
            return torch.argmax(self.dqn(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()
    
    def train(self):
        if self.batch_size > self.current:
            return
        
        states, actions, rewards, next_states, dones = self.sample()
        states = states.float().to(self.device)
        actions = actions.float().to(self.device)
        rewards = rewards.float().to(self.device)
        next_states = next_states.float().to(self.device)
        dones = dones.float().to(self.device)
        
        self.optimizer.zero_grad()
        current = self.dqn(states).gather(1, actions.long())
        target = rewards + torch.mul(self.gamma * self.dqn_target(next_states).gather(1,torch.argmax(self.dqn(next_states), dim=1).unsqueeze(1)), 1 - dones)
    
        loss = self.l1(current, target)
        loss.backward()
        self.optimizer.step()

        self.epsilon -= self.epsilon_decay
        self.epsilon = max(self.epsilon, self.epsilon_min)
        
    def store(self, state, action, reward, next_state, done):
        self.state_buffer[self.last_pos] = state
        self.action_buffer[self.last_pos] = action
        self.reward_buffer[self.last_pos] = reward
        self.next_state_buffer[self.last_pos] = next_state
        self.done_buffer[self.last_pos] = done
        
        self.last_pos = (self.last_pos + 1) % self.mem_size
        self.current = min(self.current + 1, self.mem_size)
    
    def sample(self):
        indices = random.choices(range(self.current), k=self.batch_size)
        states = self.state_buffer[indices]
        actions = self.action_buffer[indices]
        rewards = self.reward_buffer[indices]
        next_states = self.next_state_buffer[indices]
        dones = self.done_buffer[indices]      
        return states, actions, rewards, next_states, dones
        
    def update_target(self):
         self.dqn_target.load_state_dict(self.dqn.state_dict())
            
    def save_models(self):
        torch.save(self.dqn.state_dict(), "double_state_dict_model.pt")
        torch.save(self.dqn_target.state_dict(), "double_state_dict_target_model.pt")

In [3]:
#Train run
def train_agent(episodes=2000):
    
    writer = SummaryWriter()
   
    env = gym.make('ALE/MsPacman-v5', frameskip=16, obs_type='grayscale')
    env.seed(12)
    env.action_space.seed(12)
    env = Preprocess(env)
    env = ChangeAxis(env) 
     
    state_space = env.observation_space.shape
    action_space = env.action_space.n
    agent = DQN_Agent(state_space=state_space, action_space=action_space, epsilon_max=1.0, epsilon_min=0.02, rb_size=200000, batch_size=32, gamma=0.95, lr=0.00025)
    
    update_target = 200
    total_steps = 0
          
    for episode in range(episodes):
        episode_score = 0
        step = 0
        
        state = env.reset()
        state = torch.from_numpy(np.array(state)).unsqueeze(0)
        
        while True:
            action = agent.select_action(state)
            step += 1
            
            next_state, reward, done, info = env.step(int(action[0]))
            next_state = torch.from_numpy(np.array(next_state)).unsqueeze(0)
            episode_score += reward  
                    
            agent.store(state, action, reward, next_state, done)
            agent.train()
            if step % update_target == 0:
                agent.update_target()
            
            state = next_state
            
            if done:
                total_steps += step
                writer.add_scalar("Score/Step", episode_score, total_steps)
                writer.add_scalar("Score/Episode", episode_score, episode)
                writer.flush()
                break        
        
    agent.save_models()
    env.close()
    print("Done")
    writer.close()

In [4]:
train_agent(episodes=20000)

C:\Users\User\anaconda3\envs\torch_env\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Done


In [4]:
#Agent evaluation after training
def evaluate():
    env = gym.make('ALE/MsPacman-v5', frameskip=16, obs_type='grayscale')
    env = Preprocess(env)
    env = ChangeAxis(env) 

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = DQN_Network(env.observation_space.shape, env.action_space.n).to(device)
    model.load_state_dict(torch.load("double_state_dict_model.pt"))
    model.eval()

    epsilon = 0.02

    scores = []

    for i in range(30):
        episode_score = 0
        state = env.reset()
        state = torch.from_numpy(np.array(state)).unsqueeze(0)
    
        while True:
            if random.random() < epsilon:  
                action =  torch.tensor([[random.randrange(9)]])
            else:
                action = torch.argmax(model(state.to(device))).unsqueeze(0).unsqueeze(0).cpu()
            
            next_state, reward, done, info = env.step(int(action[0]))
            next_state = torch.from_numpy(np.array(next_state)).unsqueeze(0)
            episode_score += reward  
        
            state = next_state
        
            if done:
                scores.append(episode_score)
                print("Episode: {}, Score: {}" .format(i + 1, scores[-1]))
                break
            
    print("Avarage score: {}" .format(np.mean(scores)))
    print(statistics.stdev(scores))
                

In [5]:
evaluate()

C:\Users\User\anaconda3\envs\torch_env\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Episode: 1, Score: 4800.0
Episode: 2, Score: 2850.0
Episode: 3, Score: 4520.0
Episode: 4, Score: 1250.0
Episode: 5, Score: 5040.0
Episode: 6, Score: 2820.0
Episode: 7, Score: 5660.0
Episode: 8, Score: 3480.0
Episode: 9, Score: 4970.0
Episode: 10, Score: 2390.0
Episode: 11, Score: 1210.0
Episode: 12, Score: 3050.0
Episode: 13, Score: 3140.0
Episode: 14, Score: 2040.0
Episode: 15, Score: 1250.0
Episode: 16, Score: 2130.0
Episode: 17, Score: 780.0
Episode: 18, Score: 700.0
Episode: 19, Score: 3330.0
Episode: 20, Score: 1720.0
Episode: 21, Score: 2780.0
Episode: 22, Score: 3100.0
Episode: 23, Score: 4770.0
Episode: 24, Score: 3370.0
Episode: 25, Score: 1540.0
Episode: 26, Score: 4470.0
Episode: 27, Score: 2940.0
Episode: 28, Score: 5100.0
Episode: 29, Score: 4960.0
Episode: 30, Score: 2420.0
Avarage score: 3086.0
1442.12152149485
